In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')
from run_training_pipeline import *

In [3]:
import gc
import os
import sys
from datetime import datetime

import mlflow
import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from rasterio.errors import RasterioIOError
from torch.utils.data import DataLoader
from tqdm import tqdm
from yaml.loader import SafeLoader

In [5]:
os.getcwd()
os.chdir("../")

In [6]:
with open("config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

In [7]:
list_data_dir, list_masks_cloud_dir, test_dir = download_data(config)

In [18]:
test_dir = "donnees-test/classification"

In [9]:
list_data_dir = ["data/PLEIADES/2022/MARTINIQUE"]
list_masks_cloud_dir = ["data/NUAGESPLEIADES/2022/MARTINIQUE"]

In [11]:
list_output_dir = prepare_train_data(config, list_data_dir, list_masks_cloud_dir)

In [66]:
from src.classes.data.satellite_image import (SatelliteImage)

In [62]:
output_tensor = torch.randn(9, 3, 224, 224)

In [71]:
output_tensor.size()

In [184]:
pathim = "train_data-classification-PLEIADES-RIL-972-2022/images/ORT_2022_0695_1627_U20N_8Bits_002.jp2"
label = 0
        
img = SatelliteImage.from_raster(
    file_path=pathim, dep=None, date=None, n_bands=3
).array

img = np.transpose(img.astype(float), [1, 2, 0])
label = torch.tensor(label)

img = torch.tensor(img.astype(float))
img = img.permute([2, 0, 1])
#label = torch.tensor(label)

img = img.type(torch.float).unsqueeze(0)
label = label.type(torch.float)


In [185]:
print(img, label)

In [188]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision.models.resnet import ResNet50_Weights

# Load the pre-trained ResNet50 model
resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Freeze all the parameters of the pre-trained model
for param in resnet.parameters():
    param.requires_grad = False

# Get the number of input features for the last layer
num_features = resnet.fc.in_features

# Replace the last layer with a new fully connected layer for 2 classes
resnet.fc = nn.Linear(num_features, 2)

# Apply sigmoid activation to the output
resnet.softmax = nn.Softmax(dim=1)

# Forward pass through the modified model
output_tensor = img
#output_tensor = torch.randn(3, 3, 250, 250)  # Example input tensor
model_output = resnet(output_tensor)

# Get the predicted class labels
probabilities = torch.softmax(model_output, dim = 1)

probability_class_1 = probabilities[:, 1]

# Set a threshold for class prediction
threshold = 0.7

# Make predictions based on the threshold
predictions = torch.where(probability_class_1 > threshold, torch.tensor([1]), torch.tensor([0]))

predicted_classes = predictions.type(torch.float)

print(model_output)
print(probabilities)
print(probability_class_1)  # Prints the predicted class labels for each instance
print(predictions)

In [196]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.models.resnet import ResNet50_Weights

# Load the pre-trained ResNet50 model
resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Freeze all the parameters of the pre-trained model
for param in resnet.parameters():
    param.requires_grad = False

# Get the number of input features for the last layer
num_features = resnet.fc.in_features

# Replace the last layer with a new fully connected layer for 2 classes
resnet.fc = nn.Linear(num_features, 2)

# Apply sigmoid activation to the output
resnet.softmax = nn.Softmax(dim=1)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# Forward pass through the modified model
#output_tensor = img
output_tensor = torch.randn(9, 3, 250, 250)  # Example input tensor
model_output = resnet(output_tensor)

# Get the predicted class labels
probabilities = torch.softmax(model_output, dim=1)
probability_class_1 = probabilities[:, 1]

# Set a threshold for class prediction
threshold = 0.51

# Make predictions based on the threshold
predictions = torch.where(probability_class_1 > threshold, torch.tensor([1]), torch.tensor([0]))

# Convert the predictions to float type
predicted_classes = predictions.type(torch.float)

# Compute the loss
labels = torch.tensor([0,1,1,0,1,1,1,0,0])  # Example true labels
loss = criterion(model_output, labels)

# Backward pass and update the weights
optimizer.zero_grad()
loss.backward()
optimizer.step()

print(model_output)
print(probabilities)
print(probability_class_1)  # Prints the predicted class probabilities for each instance
print(predictions)  # Prints the predicted class labels for each instance
print(optimizer)
print(loss)

In [197]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision.models.resnet import ResNet50_Weights

# Load the pre-trained ResNet50 model
resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Freeze all the parameters of the pre-trained model
for param in resnet.parameters():
    param.requires_grad = False

# Get the number of input features for the last layer
num_features = resnet.fc.in_features

# Replace the last layer with a new fully connected layer for 2 classes
resnet.fc = nn.Linear(num_features, 2)

# Apply sigmoid activation to the output
resnet.sigmoid = nn.Sigmoid()

# Forward pass through the modified model
output_tensor = torch.randn(3, 3, 250, 250)  # Example input tensor
model_output = resnet(output_tensor)

# Get the predicted class labels
predicted_labels = (torch.max(model_output, dim = 1).indices).clone().detach()

print(model_output)
print(predicted_labels)  # Prints the predicted class labels for each instance


In [147]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load the pre-trained ResNet50 model
resnet = models.resnet50(pretrained=True)

# Freeze all the parameters of the pre-trained model
for param in resnet.parameters():
    param.requires_grad = False

# Get the number of input features for the last layer
num_features = resnet.fc.in_features

# Replace the last layer with a new fully connected layer for binary classification
resnet.fc = nn.Linear(num_features, 1)

# Apply sigmoid activation to the output
resnet.sigmoid = nn.Sigmoid()

# Create a batch of input tensors
batch_size = 9
input_tensor = torch.randn(batch_size, 3, 224, 224)  # Example input tensor

# Create a batch of target tensors (assuming binary classification)
target_tensor = torch.randint(0, 2, (batch_size, 1)).float()  # Example target tensor

# Forward pass through the model
output = resnet(input_tensor)

# Apply sigmoid activation to the output
output = resnet.sigmoid(output)

# Reshape the output tensor to match the target tensor shape
output = output.view(-1, 1)

# Create the BCE loss criterion
criterion = nn.BCELoss()

# Compute the BCE loss
loss = nn.BCELoss()(output, target_tensor)

print(loss)  # Prints the computed loss


In [ ]:
from run_training_pipeline import run_pipeline
remote_server_uri = "https://projet-slums-detection-874257.user.lab.sspcloud.fr"
experiment_name = "classification"
run_name = "patch_binary_patch100_0.53_validation"
run_pipeline(remote_server_uri, experiment_name, run_name)

In [203]:
import sys
sys.path.append('../src')

import gc
import os
import sys
from datetime import datetime

import mlflow
import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from rasterio.errors import RasterioIOError
from torch.utils.data import DataLoader
from tqdm import tqdm
from yaml.loader import SafeLoader

import train_pipeline_utils.handle_dataset as hd
from classes.data.satellite_image import SatelliteImage
from classes.labelers.labeler import BDTOPOLabeler, RILLabeler
from classes.optim.losses import CrossEntropySelfmade
from torch.nn import CrossEntropyLoss
from classes.optim.optimizer import generate_optimization_elements
from data.components.dataset import PleiadeDataset
from data.components.classification_patch import PatchClassification
from models.components.segmentation_models import DeepLabv3Module
from models.segmentation_module import SegmentationModule
from train_pipeline_utils.download_data import load_satellite_data, load_donnees_test
from train_pipeline_utils.handle_dataset import (
    generate_transform,
    select_indices_to_split_dataset
)

from train_pipeline_utils.prepare_data import(
    filter_images,
    label_images,
    save_images_and_masks,
    check_labelled_images
)

from classes.data.satellite_image import SatelliteImage
from classes.data.labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils.utils import update_storage_access, split_array, remove_dot_file
from rasterio.errors import RasterioIOError
from classes.optim.evaluation_model import evaluer_modele_sur_jeu_de_test_segmentation_pleiade


In [211]:
os.getcwd()
#os.chdir("src")

In [213]:
with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)
config

In [214]:
def download_data(config):
    """
    Downloads data based on the given configuration.

    Args:
        config: a dictionary representing the 
        configuration information for data download.

    Returns:
        A list of output directories for each downloaded dataset.
    """

    print("Entre dans la fonction download_data")
    config_data = config["donnees"]
    list_output_dir = []
    list_masks_cloud_dir = []
    
    years = config_data["year"]
    deps = config_data["dep"]
    src = config_data["source train"]

    for year, dep in zip(years, deps):
        # year, dep = years[0], deps[0]
        if src == "PLEIADES":
            cloud_dir = load_satellite_data(year, dep, "NUAGESPLEIADES")
            list_masks_cloud_dir.append(cloud_dir)

        output_dir = load_satellite_data(year, dep, src)
        list_output_dir.append(output_dir)
    
    print("chargement des données test")
    test_dir = load_donnees_test(type=config["donnees"]["task"])

    return list_output_dir, list_masks_cloud_dir, test_dir

In [25]:
list_output_dir, list_masks_cloud_dir, test_dir = download_data(config)

In [215]:
def label_images2(list_images, labeler):
    """
    labels the images according to type of labeler desired.

    Args:
        list_images : the list containing the splitted and filtered data \
            to be labeled.
        labeler : a Labeler object representing the labeler \
            used to create segmentation labels.

    Returns:
        list[SatelliteImage] : the list containing the splitted and \
            filtered data with a not-empty mask and the associated masks.
    """

    # print("Entre dans la fonction label_images")
    list_masks = []
    list_filtered_splitted_labeled_images = []

    for satellite_image in list_images:
        mask = labeler.create_segmentation_label(satellite_image)
        list_filtered_splitted_labeled_images.append(satellite_image)
        if np.sum(mask) >=1:
            list_masks.append(1)
        else:
            list_masks.append(0)

    # print(
    #     "Nombre d'images labelisées : ",
    #     len(list_filtered_splitted_labeled_images),
    #     ", Nombre de masques : ",
    #     len(list_masks),
    # )
    return list_filtered_splitted_labeled_images, list_masks

def save_images_and_masks2(list_images, list_masks, output_directory_name):
    """
    write the couple images/masks into a specific folder.

    Args:
        list_images : the list containing the splitted and filtered data \
            to be saved.
        list_masks : the list containing the masks to be saved.
        a string representing the name of the output \
            directory where the split images and their masks should be saved.

    Returns:
        str: The name of the output directory.
    """
    
    # print("Entre dans la fonction save_images_and_masks")
    output_images_path = output_directory_name + "images"
    output_masks_path = output_directory_name + "labels"

    
    csv_file_path = output_masks_path + "/" + 'fichierlabeler.csv'

    i = 0
    for image, mask in zip(list_images, list_masks):
        filename = image.filename.split(".")[0] + "_" + "{:03d}".format(i)
        i += 1
    
        try:
            image.to_raster(output_images_path, filename + ".jp2", "jp2", None)
    
        except rasterio._err.CPLE_AppDefinedError:
            print("Writing error:", image.filename)
            continue
    
        # Create the csv file if it does not exist
        if not os.path.isfile(csv_file_path):
            with open(csv_file_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Path_image', 'Classification'])
                writer.writerow([filename, mask])
    
        # Open it if it exists
        else:
            with open(csv_file_path, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([filename, mask])
        
        #print("Les nouvelles lignes ont été ajoutées avec succès.")




    return None

In [216]:
def prepare_train_data(config, list_data_dir, list_masks_cloud_dir):
    """
    Preprocesses and splits the raw input images 
    into tiles and corresponding masks, 
    and saves them in the specified output directories. 
    
    Args:
        config: A dictionary representing the configuration settings.
        list_data_dir: A list of strings representing the paths
        to the directories containing the raw input image files.
    
    Returns:
        A list of strings representing the paths to 
        the output directories containing the 
        preprocessed tile and mask image files.
    """

    print("Entre dans la fonction prepare_data")
    config_data = config["donnees"]

    years = config_data["year"]
    deps = config_data["dep"]
    src = config_data["source train"]
    labeler = config_data["type labeler"]

    list_output_dir = []

    for i, (year, dep) in enumerate(zip(years, deps)):
        # i, year , dep = 0,years[0],deps[0]
        output_dir = (
            "train_dataaa"
            + "-"
            + src
            + "-"
            + labeler
            + "-"
            + dep
            + "-"
            + str(year)
            + "/"
        )

        date = datetime.strptime(str(year) + "0101", "%Y%m%d")

        if labeler == "RIL":
            buffer_size = config_data["buffer size"]
            labeler = RILLabeler(date, dep=dep, buffer_size=buffer_size)

        elif labeler == "BDTOPO":
            labeler = BDTOPOLabeler(date, dep=dep)

        if not check_labelled_images(output_dir):

            list_name_cloud = []
            if src == "PLEIADES":
                cloud_dir = list_masks_cloud_dir[i]
                list_name_cloud = [path.split("/")[-1].split(".")[0] for path in os.listdir(cloud_dir)]
            
            dir = list_data_dir[i]
            list_path = [dir + "/" + filename for filename in os.listdir(dir)]

            for path in tqdm(list_path):
                # path = list_path[0]
                # path  = dir + "/"+ "ORT_2022_0691_1641_U20N_8Bits.jp2"
                try:
                    si = SatelliteImage.from_raster(
                        file_path=path,
                        dep=dep,
                        date=date,
                        n_bands=config_data["n bands"],
                    )
                    
                except RasterioIOError:
                    print("Erreur de lecture du fichier " + path)
                    continue

                else:
                    filename = path.split("/")[-1].split(".")[0] 
                    list_splitted_mask_cloud = None

                    if filename in list_name_cloud:
                        mask_full_cloud = np.load(cloud_dir + "/" + filename + ".npy")
                        list_splitted_mask_cloud = split_array(mask_full_cloud, config_data["tile size"])
                        
                    list_splitted_images = si.split(config_data["tile size"]) 
                    
                    list_filtered_splitted_images = filter_images(
                        config_data["source train"],
                        list_splitted_images,
                        list_splitted_mask_cloud 
                    )

                    (
                        list_filtered_splitted_labeled_images,
                        list_masks,
                    ) = label_images2(list_filtered_splitted_images, labeler)

                    save_images_and_masks2(
                        list_filtered_splitted_labeled_images,
                        list_masks,
                        output_dir,
                    )

        list_output_dir.append(output_dir)
        nb = len(os.listdir(output_dir + "/images"))
        print(str(nb) + " couples images/masques retenus")

    return list_output_dir

In [217]:
list_data_dir = ['../data/PLEIADES/2022/MARTINIQUE/']

In [52]:
os.getcwd()
os.chdir("..")

In [218]:
list_output_dir

In [219]:
list_output_dir = prepare_train_data(config, list_data_dir, list_masks_cloud_dir)

In [23]:
import csv 

csv_file_path = "src/train_data-classification-PLEIADES-RIL-972-2022" + "/labels/" + "fichierlabeler.csv"
zero_count = 0
one_count = 0

with open(csv_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        classification = row['Classification']
        if classification == '0':
            zero_count += 1
        elif classification == '1':
            one_count += 1

prop_ones = one_count/(zero_count + one_count)
print(zero_count, one_count, prop_ones)

In [ ]:
csv_file_path = "chemin_vers_le_fichier.csv"
selected_paths = []

# Lire le fichier CSV et collecter les chemins correspondants à classification == 0 et classification == 1
with open(csv_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    classification_0_paths = []
    classification_1_paths = []

    for row in reader:
        classification = int(row['Classification'])
        path = row['Path']

        if classification == 0:
            classification_0_paths.append(path)
        elif classification == 1:
            classification_1_paths.append(path)

    # Sélectionner au hasard une proportion 50/50 de chemins
    selected_paths.extend(random.sample(classification_0_paths, k=len(classification_1_paths)))
    selected_paths.extend(classification_1_paths)

# Afficher les chemins sélectionnés
print("Chemins sélectionnés :")
for path in selected_paths:
    print(path)

In [220]:
def prepare_test_data(config,test_dir):

    if config["donnees"]["source train"] == "PLEIADES":

        images_path = test_dir + "/images"
        labels_path = test_dir + "/masks"

        list_name_image = os.listdir(images_path)
        list_name_label = os.listdir(labels_path)

        list_name_image = np.sort(remove_dot_file(list_name_image))
        list_name_label = np.sort(remove_dot_file(list_name_label))

        list_images_path = [images_path + "/" + name for name in list_name_image]
        list_labels_path = [labels_path + "/" + name for name in list_name_label]
        
        output_test = "../test-data"
        output_images_path = output_test + "/images"
        output_labels_path = output_test + "/labels"

        n_bands = config["donnees"]["n bands"]
        tile_size = config["donnees"]["tile size"]
        
        if not os.path.exists(output_labels_path):
            os.makedirs(output_labels_path)
        else:
            return None

        for image_path, label_path, name in zip(
            list_images_path,
            list_labels_path,
            list_name_image
            ):

            si = SatelliteImage.from_raster(
                file_path=image_path, dep=None, date=None, n_bands=n_bands
            )
            mask = np.load(label_path)

            lsi = SegmentationLabeledSatelliteImage(si, mask, "", "")
            list_lsi = lsi.split(tile_size)

            for i, lsi in enumerate(list_lsi):
                file_name_i = name.split(".")[0] + "_" + "{:03d}".format(i)
                
                lsi.satellite_image.to_raster(
                    output_images_path, file_name_i + ".jp2"
                    )
                np.save(output_labels_path + "/" + file_name_i + ".npy", lsi.label)

In [234]:
def instantiate_dataset2(config, list_path_images, list_labels):
    """
    Instantiates the appropriate dataset object
    based on the configuration settings.
    
    Args:
        config: A dictionary representing the configuration settings.
        list_path_images: A list of strings representing
        the paths to the preprocessed tile image files.
        list_path_labels: A list of strings representing
        the paths to the corresponding preprocessed mask image files.
    
    Returns:
        A dataset object of the specified type.
    """
    dataset_dict = {"PLEIADE": PleiadeDataset, "CLASSIFICATION" : PatchClassification}
    dataset_type = config["donnees"]["dataset"]

    # inqtanciation du dataset comple
    if dataset_type not in dataset_dict:
        raise ValueError("Invalid dataset type")
    else:
        dataset_select = dataset_dict[dataset_type]

        full_dataset = dataset_select(
            list_path_images, list_labels, config["donnees"]["n channels train"]
        )

    return full_dataset



In [232]:
def instantiate_dataloader2(config, list_output_dir):
    """
    Instantiates and returns the data loaders for
    training, validation, and testing datasets.
    
    Args:
    - config (dict): A dictionary containing the configuration parameters
    for data loading and processing.
    - list_output_dir (list): A list of strings containing the paths to
    the directories that contain the training data.
    
    Returns:
    - train_dataloader (torch.utils.data.DataLoader):
    The data loader for the training dataset.
    - valid_dataloader (torch.utils.data.DataLoader): 
    The data loader for the validation dataset.
    - test_dataloader (torch.utils.data.DataLoader): 
    The data loader for the testing dataset.
    
    The function first generates the paths for the image and label data
    based on the data source (Sentinel, PLEIADES) vs pre-annotated datasets.
    It then instantiates the required dataset class
    (using the `intantiate_dataset` function) and splits the full dataset
    into training and validation datasets based on the validation proportion
    specified in the configuration parameters.
    
    Next, the appropriate transformations are applied to the training
    and validation datasets using the `generate_transform` function.
    
    Finally, the data loaders for the training and validation datasets 
    are created using the `DataLoader` class from the PyTorch library, 
    and the data loader for the testing dataset is set to `None`.
    """
    # génération des paths en fonction du type de Données
    # (Sentinel, PLEIADES) VS Dataset préannotés

    if config["donnees"]["source train"] in ["PLEIADES", "SENTINEL2"]:
        for dir in list_output_dir:
            # dir = list_output_dir[0]
            labels = os.listdir(dir + "/labels") 
            if labels[0][0] == ".":
                del(labels[0])

            images = os.listdir(dir + "/images")
            
            liste_masks = []
            
            with open(dir + "/labels/" + labels[0], 'r') as csvfile:
                reader = csv.reader(csvfile)
                
                # Ignorer l'en-tête du fichier CSV s'il y en a un
                next(reader)
                
                # Parcourir les lignes du fichier CSV et extraire la deuxième colonne
                for row in reader:
                    image_path = row[0]
                    mask = row[1] # Index 1 correspond à la deuxième colonne (index 0 pour la première)
                    liste_masks.append([image_path, mask])

            liste_masks = sorted(liste_masks, key=lambda x: x[0])
            liste_masks = np.array([sous_liste[1] for sous_liste in liste_masks])

            
            list_path_images = np.sort([dir + "/images/" + name for name in images])

    train_idx, val_idx = select_indices_to_split_dataset(
        len(list_path_images),
        config["optim"]["val prop"]
    )
    
    # récupération de la classe de Dataset souhaitée
    train_dataset = instantiate_dataset2(
        config, list_path_images[train_idx], liste_masks[train_idx]
    )

    valid_dataset = instantiate_dataset2(
        config, list_path_images[val_idx], liste_masks[val_idx]
    )

    # on applique les transforms respectives
    augmentation = config["donnees"]["augmentation"]
    tile_size = config["donnees"]["tile size"]

    if config["donnees"]["source train"] == "PLEIADES":
        t_aug, t_preproc = generate_transform(tile_size, augmentation)
    else:
        t_aug, t_preproc = None, None

    train_dataset.transforms = t_aug
    valid_dataset.transforms = t_preproc

    # création des dataloader
    batch_size = config["optim"]["batch size"]

    train_dataloader, valid_dataloader = [
        DataLoader(
            ds,
            batch_size=batch_size,
            shuffle=boolean,
            num_workers=2,
        )
        for ds, boolean in zip([train_dataset, valid_dataset], [True, False])
    ]

    # Gestion datset test
    output_test = "../test-data"
    output_images_path = output_test + "/images/"
    output_labels_path = output_test + "/labels/"

    list_name_image = os.listdir(output_images_path)
    list_name_label = os.listdir(output_labels_path)

    list_path_images = np.sort([output_images_path + name_image for name_image in list_name_image])
    list_path_labels = np.sort([output_labels_path + name_label for name_label in list_name_label])

    dataset_test = instantiate_dataset3(
        config, list_path_images, list_path_labels
    )
    
    dataset_test.transforms = t_preproc
    
    batch_size_test = config["optim"]["batch size test"]
    test_dataloader = DataLoader(
            dataset_test,
            batch_size=batch_size_test,
            shuffle=False,
            num_workers=2,
        )
    
    return train_dataloader, valid_dataloader, test_dataloader


In [231]:
def instantiate_dataset3(config, list_path_images, list_labels):
    """
    Instantiates the appropriate dataset object
    based on the configuration settings.
    
    Args:
        config: A dictionary representing the configuration settings.
        list_path_images: A list of strings representing
        the paths to the preprocessed tile image files.
        list_path_labels: A list of strings representing
        the paths to the corresponding preprocessed mask image files.
    
    Returns:
        A dataset object of the specified type.
    """
    dataset_dict = {"PLEIADE": PleiadeDataset, "CLASSIFICATION" : PatchClassification}
    dataset_type = "PLEIADE"

    # inqtanciation du dataset comple
    if dataset_type not in dataset_dict:
        raise ValueError("Invalid dataset type")
    else:
        dataset_select = dataset_dict[dataset_type]

        full_dataset = dataset_select(
            list_path_images, list_labels, config["donnees"]["n channels train"]
        )

    return full_dataset

In [339]:
list_output_dir =['../train_data-classification-PLEIADES-RIL-972-2022']
train_dataloader, valid_dataloader, test_dataloader = instantiate_dataloader2(config, list_output_dir)

In [ ]:
images, labels, dic =next(iter(train_dataloader))
model = instantiate_model(config)

In [327]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score

# Générer des données de classification
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score

# Générer des données de classification
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner le modèle de classification
model = LogisticRegression()
model.fit(X_train, y_train)

# Prédire les probabilités des classes positives
y_probs = model.predict_proba(X_test)[:, 1]

# Calculer le taux de vrais positifs (TPR), le taux de faux positifs (FPR) et les seuils
fpr, tpr, thresholds = roc_curve(y_test, y_probs)

# Calculer l'aire sous la courbe ROC (AUC)
auc = roc_auc_score(y_test, y_probs)

# Tracer la courbe ROC
plt.figure()
plt.plot(fpr, tpr, label='Courbe ROC (AUC = {:.2f})'.format(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Aléatoire')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc='lower right')
plt.show()


In [257]:
images, labels, dic =next(iter(train_dataloader))
model = instantiate_model(config)

In [265]:
target


In [288]:
targets.unsqueeze(1)
labels.unsqueeze(1)

In [276]:
targets_one_hot = torch.zeros(num_samples, num_classes)

targets_one_hot.scatter_(1, targets.unsqueeze(1), 1)

print(target,targets_one_hot)


In [274]:
target
# Create random predictions (logits) and target labels
num_samples = 10
num_classes = 2 
predictions = torch.randn(num_samples, num_classes)

targets = torch.randint(0, 2, (num_samples,))
targets
predictions

output

In [286]:
targets_one_hot = torch.zeros(labels.shape[0], 2)
targets_one_hot
targets_one_hot.scatter_(1, labels.unsqueeze(1), 1)

In [296]:
predictions


In [317]:
output
probability_class_1 = output[:, 1]
probability_class_1

# Set a threshold for class prediction
threshold = 0.53

# Make predictions based on the threshold
predictions = torch.where(probability_class_1 < threshold, torch.tensor([1]), torch.tensor([0]))


In [313]:
probability_class_1

In [311]:
labels

In [318]:
predictions

In [321]:
predicted_classes = predictions.type(torch.float)

misclassified_percentage = (predicted_classes != labels).float().mean()

In [322]:
misclassified_percentage

In [325]:
probability_class_1 = output[:, 1]


# Set a threshold for class prediction
threshold = 0.53

# Make predictions based on the threshold
predictions = torch.where(probability_class_1 < threshold, torch.tensor([1]), torch.tensor([0]))


predicted_classes = predictions.type(torch.float)

misclassified_percentage = (predicted_classes != labels).float().mean()

In [326]:
misclassified_percentage

In [301]:

target = labels
output  = model(images)
print(labels.shape,output.shape)
labels
output
labels = labels.long()
targets_one_hot = torch.zeros(labels.shape[0], 2)
targets_one_hot = targets_one_hot.scatter_(1, labels.unsqueeze(1), 1)


print(targets_one_hot) 
print(output)

from torch import nn
criterion = nn.BCELoss()

criterion(output,targets_one_hot)




In [298]:
import torch
import torch.nn as nn

# Assuming you have a binary classification problem with 3 samples and 2 classes
num_samples = 3
num_classes = 2

# Create random predictions (logits) and target labels
predictions = torch.randn(num_samples, num_classes)
targets = torch.randint(0, 2, (num_samples,))

# Define the BCE loss criterion
criterion = nn.BCEWithLogitsLoss()

# Convert the targets to one-hot encoding
targets_one_hot = torch.zeros(num_samples, num_classes)
targets_one_hot.scatter_(1, targets.unsqueeze(1), 1)

# Calculate the loss
loss = criterion(predictions, targets_one_hot)

print("Predictions:", predictions)
print("Targets (one-hot):", targets_one_hot)
print("BCE Loss:", loss.item())

In [143]:
import torch
import torch.nn as nn
import torch.optim as optim

In [154]:
from src.models.classification_module import ClassificationModule

In [159]:
from src.models.components.classification_models import BinaryClassificationModule

In [244]:
def instantiate_model(config):
    """
    Instantiate a module based on the provided module type.

    Args:
        module_type (str): Type of module to instantiate.

    Returns:
        object: Instance of the specified module.
    """
    module_type = config["optim"]["module"]
    module_dict = {"deeplabv3": DeepLabv3Module, "resnet50" :  ResNet50Module}
    nchannel = config["donnees"]["n channels train"]

    if module_type not in module_dict:
        raise ValueError("Invalid module type")

    if module_type == "deeplabv3":
        return module_dict[module_type](nchannel)
    else:
        return module_dict[module_type]()


def instantiate_loss(config):
    """
    intantiates an optimizer object with the parameters 
    specified in the configuration file.

    Args:
        model: A PyTorch model object.
        config: A dictionary object containing the configuration parameters.

    Returns:
        An optimizer object from the `torch.optim` module.
    """
    loss_type = config["optim"]["loss"]
    loss_dict = {
                "crossentropy": CrossEntropyLoss,
                "crossentropyselmade": CrossEntropySelfmade,
                "lossbinaire" : nn.BCELoss
                }

    if loss_type not in loss_dict:
        raise ValueError("Invalid loss type")
    else:
        return loss_dict[loss_type]()


def instantiate_lightning_module(config, model):
    """
    Create a PyTorch Lightning module for segmentation
    with the given model and optimization configuration.

    Args:
        config (dict): Dictionary containing the configuration
        parameters for optimization.
        model: The PyTorch model to use for segmentation.

    Returns:
        A PyTorch Lightning module for segmentation.
    """
    list_params = generate_optimization_elements(config)

    lightning_module = ClassificationModule(
                    model=model,
                    loss=nn.CrossEntropyLoss(),
                    optimizer=optim.Adam,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                )
    return lightning_module


def instantiate_trainer(config, lightning_module):
    """
    Create a PyTorch Lightning module for segmentation with 
    the given model and optimization configuration.

    Args:
        config (dict): Dictionary containing the configuration
        parameters for optimization.
        model: The PyTorch model to use for segmentation.

    Returns:
        SegmentationModule: A PyTorch Lightning module for segmentation.
    """
    # def callbacks
    checkpoint_callback = ModelCheckpoint(
        monitor="validation_loss", save_top_k=1, save_last=True, mode="min"
    )

    checkpoint_callback_IOU = ModelCheckpoint(
        monitor="validation_IOU", save_top_k=1, save_last=True, mode="max"
    )
    early_stop_callback = EarlyStopping(
        monitor="validation_loss", mode="min", patience=20
    )
    lr_monitor = LearningRateMonitor(logging_interval="step")
    list_callbacks = [lr_monitor, checkpoint_callback, early_stop_callback, checkpoint_callback_IOU]

    strategy = "auto"

    trainer = pl.Trainer(
        callbacks=list_callbacks,
        max_epochs=config["optim"]["max epochs"],
        num_sanity_val_steps=2,
        strategy=strategy,
        log_every_n_steps=2,
        accumulate_grad_batches=config["optim"]["accumulate batch"]
    )

    return trainer

In [243]:
with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)
config

In [ ]:
def run_pipeline(remote_server_uri, experiment_name, run_name):

    model = instantiate_model(config)

    light_module = instantiate_lightning_module(config, model)
    trainer = instantiate_trainer(config, light_module)

    torch.cuda.empty_cache()
    gc.collect()


    if config["mlflow"]:

        update_storage_access()
        os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://minio.lab.sspcloud.fr"
        mlflow.end_run()
        mlflow.set_tracking_uri(remote_server_uri)
        mlflow.set_experiment(experiment_name)
        # mlflow.pytorch.autolog()

        with mlflow.start_run(run_name=run_name):
            mlflow.autolog()    
            mlflow.log_artifact(
                "config.yml",
                artifact_path="config.yml"
            )
            trainer.fit(light_module, train_dataloader, valid_dataloader)
            model = light_module.model
            tile_size = config["donnees"]["tile size"]
            batch_size_test = config["optim"]["batch size test"]
            
            if config["donnees"]["source train"] == "PLEIADES":

                light_module_checkpoint = light_module.load_from_checkpoint(
                    loss = instantiate_loss(config),
                    checkpoint_path=trainer.checkpoint_callback.best_model_path, # je créé un module qui charge
                    model=light_module.model,
                    optimizer=light_module.optimizer,
                    optimizer_params=light_module.optimizer_params,
                    scheduler=light_module.scheduler,
                    scheduler_params=light_module.scheduler_params,
                    scheduler_interval=light_module.scheduler_interval

                )
                evaluer_modele_sur_jeu_de_test_segmentation_pleiade(
                        test_dataloader,
                        model,
                        tile_size,
                        batch_size_test,
                        config["mlflow"]
                    )
    
    else:
        trainer.fit(light_module, train_dataloader, valid_dataloader)
        model = light_module.model
        tile_size = config["donnees"]["tile size"]
        batch_size_test = config["optim"]["batch size test"]
        
        if config["donnees"]["source train"] == "PLEIADES":

            light_module_checkpoint = light_module.load_from_checkpoint(
                loss = instantiate_loss(config),
                checkpoint_path=trainer.checkpoint_callback.best_model_path, # je créé un module qui charge
                model=light_module.model,
                optimizer=light_module.optimizer,
                optimizer_params=light_module.optimizer_params,
                scheduler=light_module.scheduler,
                scheduler_params=light_module.scheduler_params,
                scheduler_interval=light_module.scheduler_interval

                )
            
            model = light_module_checkpoint.model   

            evaluer_modele_sur_jeu_de_test_segmentation_pleiade(
                    test_dataloader,
                    model,
                    tile_size,
                    batch_size_test,
                    config["mlflow"]
                )
        # trainer.test(light_module, test_dl)


if __name__ == "__main__":
    # MLFlow params
    remote_server_uri = "https://projet-slums-detection-874257.user.lab.sspcloud.fr"
    experiment_name = "segmentation"
    run_name = "essai_patch_binaire"
    run_pipeline(remote_server_uri, experiment_name, run_name)

In [ ]:
import sys
sys.path.append('../src')
from run_training_pipeline2 import *



In [10]:
import matplotlib.pyplot as plt

In [11]:
mask = np.load("../test-data/labels/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bits_000.npy")
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax.imshow(mask, cmap="gray")
ax.set_title("mask")
ax.axis('off')
plt.show()